# Gurobipy

- The Gurobi Optimizer is a mathematical optimization software library for solving mixed-integer linear and quadratic optimization problems.

- This package comes with a trial license that allows you to solve problems of limited size. 

- As a student or staff member of an academic institution you qualify for a free, full product license. 

- For more information about academic licenses, see: https://www.gurobi.com/academia/academic-program-and-licenses/

## Installing Gurobi for Python

In [ ]:
#pip install gurobipy

## Example: MIP (Binary problem)

Simple example:
- You want to decide about three activities (do or don‘t do) and aim for maximum value
- You need to choose at least activity 1 or 2 (or both)
- The total time limit is 4 hours
    - Activity 1 takes 1 hours
    - Activity 2 takes 2 hours
    - Activity 3 takes 4 hours
- Activity 3 is worth twice as much as 1 and 2

This can be modeled as a mixed-integer linear program
- Binary variables x,y,z for activities 1,2,3
- Linear constraint for time limit
- Linear constraint for condition (1 or 2)


- Formulação do problema:
$$
\begin{align}
max  \; & x + y + 2z \\
& x + 2 y + 3 z <= 4 \\
& x +   y       >= 1 \\
& x, y, z \in \{0,1\}
\end{align}
$$

## Python interface

### Importing the Gurobi functions and classes

In [1]:
import gurobipy as gp
from gurobipy import GRB

### Creating the model

In [10]:
# Create a new model
model1 = gp.Model("mip") 

### Adding variables to the model

In [16]:
# Create variables
x = model1.addVar(vtype=GRB.BINARY, name="x")
y = model1.addVar(vtype=GRB.BINARY, name="y")
z = model1.addVar(vtype=GRB.BINARY, name="z")

### Adding constraints to the model

In [17]:
# Add constraint: x + 2 y + 3 z <= 4
model1.addConstr(x + 2 * y + 3 * z <= 4, "c0")

# Add constraint: x + y >= 1
model1.addConstr(x + y >= 1, "c1")

<gurobi.Constr *Awaiting Model Update*>

### Setting the objective

In [18]:
# Set objective
model1.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

### Save model(.lp or .mps)

In [19]:
# save model for inspection
model1.write('model1.mps')

### Optimizing the model

In [20]:
# Optimize model
model1.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 6 columns and 10 nonzeros
Model fingerprint: 0x8b66ea02
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 4 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


### Reporting the results

In [ ]:
for v in model1.getVars():
    print('%s = %g' % (v.VarName, v.X))

print('Obj: %g' % model1.ObjVal)

print('Runtime: %g' % model1.Runtime)

## Python matrix interface

### Importing the Gurobi functions and classes

In [24]:
import gurobipy as gp
from gurobipy import GRB

import numpy as np
import scipy.sparse as sp

### Creating the model

In [25]:
# Create a new model
model2 = gp.Model("mip2") 

### Adding variables to the model

In [26]:
# Create variables
x = model2.addMVar(shape=3, vtype=GRB.BINARY, name="x")

### Adding constraints to the model

In [27]:
# Build (sparse) constraint matrix
val = np.array([1.0, 2.0, 3.0, -1.0, -1.0])
row = np.array([0, 0, 0, 1, 1])
col = np.array([0, 1, 2, 0, 1])

A = sp.csr_matrix((val, (row, col)), shape=(2, 3))

# Build rhs vector
rhs = np.array([4.0, -1.0])

# Add constraints
model2.addConstr(A @ x <= rhs, name="c")

model2.update

<bound method Model.update of <gurobi.Model Continuous instance mip2: 0 constrs, 0 vars, Parameter changes: Username=(user-defined)>>

### Setting the objective

In [28]:
# Set objective
obj = np.array([1.0, 1.0, 2.0])
model2.setObjective(obj @ x, GRB.MAXIMIZE)

### Save model(.lp or .mps)

In [29]:
# save model for inspection
model2.write('model2.lp')

### Optimizing the model

In [30]:
# Optimize model
model2.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x8d4960d3
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


### Reporting the results

In [31]:
print(f"Optimal objective value: {model2.objVal}")
print(f"Solution values: x = {x.X}")
print(f"Runtime: {model2.Runtime}")


Optimal objective value: 3.0
Solution values: x = [1. 0. 1.]
Runtime: 0.010692119598388672


## Example: Resource Assignment Problem formulation

### Problem statement

Consider three job positions: Tester, Java-Developer, and Architect.

Consider three resources: Carlos, Joe, and Monika.

Determine an assignment that ensures that each job is fulfilled and each resource is assigned to at most one job in order to maximize the total matching scores of the assignments.

### Data

The ability to perform each of the jobs by each of the resources is illustrated by the following matching scores table:

![Resource Allocation Problem Data Image](rap_data.png)


**Assumption**: Only one resource can be assigned to a job, and at most one job can be assigned to a resource.

### Decision variables

The decision variable $x_{r,j} = 1$ represents that resource $r$ is assigned to job $j$, and 0 otherwise, for  $r=1,2,3$ and $j=1,2,3$.

### Constraints: jobs and resources

**Jobs constraints**

For each job $j=1,2,3$, exactly one resource from $r=1,2,3$ must be assigned.

Constraint (Tester=1): $x_{1,1} + x_{2,1} + x_{3,1} = 1$

Constraint (Java-Developer=2): $x_{1,2} + x_{2,2} + x_{3,2} = 1$

Constraint (Architect=3): $x_{1,3} + x_{2,3} + x_{3,3} = 1$

**Resources constraints**

For each resource = $r=1,2,3$, at most one job from $r=1,2,3$ can be assigned.

Constraint (Carlos=1): $x_{1,1} + x_{1,2} + x_{1,3}  \leq 1$

Constraint (Joe=2): $x_{2,1} + x_{2,2} + x_{2,3}  \leq 1$

Constraint (Monika=3): $x_{2,1} + x_{2,2} + x_{2,3}  \leq 1$

### Objective function

The objective function is to maximize the total matching score of the assignments while satisfying the jobs and resources constraints.

$$
\max \; 
(53x_{1,1} + 80x_{2,1} + 53x_{3,1}) 
+ (27x_{1,2} + 47x_{2,2} + 73x_{3,2})
+ (13x_{1,3} + 67x_{2,3} + 47x_{3,3})
$$

### Importing the Gurobi functions and classes

In [ ]:
# import gurobi library
import gurobipy as gp
from gurobipy import GRB

### Data

The list R contains the names of the three resources: Carlos, Joe, and Monika. 

The list J contains the names of the job positions: tester, java-developer, and architect

In [ ]:
# resources and jobs sets
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

$r \in R$ index and set of resources.

$j \in J$ index and set of Jobs.

The following “multidict” function describes the matching score associated with each possible combination of a resource and job

In [ ]:
# matching score data
combinations, ms = gp.multidict({
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'JavaDeveloper'): 27,
    ('Carlos', 'Architect'): 13,
    ('Joe', 'Tester'): 80,
    ('Joe', 'JavaDeveloper'): 47,
    ('Joe', 'Architect'): 67,
    ('Monika', 'Tester'): 53,
    ('Monika', 'JavaDeveloper'): 73,
    ('Monika', 'Architect'): 47
})

### Creating the model

The following function generates an empty model object “model” and takes the string “RAP” model name as its argument.

In [ ]:
# Declare and initialize model
model3 = gp.Model('rap')

### Adding variables to the model

The decision variable $x_{r,j} = 1$ represents that resource $r$ is assigned to job $j$, and 0 otherwise, for $r \in R$ and $j \in J $.

The “addVars()” method defines the decision variables of the model object “model”.  

In [ ]:
# Create decision variables for the RAP model
x = model3.addVars(combinations, vtype=GRB.BINARY, name="x")

### Adding constraints to the model: job and resource

**Job constraints**

For each job $j=1,2,3$, exactly one resource from $r=1,2,3$ must be assigned.

Constraint (Tester=1): $x_{1,1} + x_{2,1} + x_{3,1} = 1$

Constraint (Java-Developer=2): $x_{1,2} + x_{2,2} + x_{3,2} = 1$

Constraint (Architect=3): $x_{1,3} + x_{2,3} + x_{3,3} = 1$

The “addConstrs()” method defines the constraints of the model object “model”. 


In [ ]:
# create jobs  constraints
job = model3.addConstrs((x.sum('*',j) == 1 for j in J), 'job')

$$
\sum_{r \: \in \: R} x_{r,j} = 1 \; \; \; \forall \; j \in J
$$

**Resource constraints**

For each resource $r=1,2,3$, at most one job from $r=1,2,3$ can be assigned.

Constraint (Carlos=1): $x_{1,1} + x_{1,2} + x_{1,3}  \leq 1$

Constraint (Joe=2): $x_{2,1} + x_{2,2} + x_{2,3}  \leq 1$

Constraint (Monika=3): $x_{3,1} + x_{3,2} + x_{3,3}  \leq 1$

The “addConstrs()” method defines the constraints of the model object “model”. 

In [ ]:
# create resources constraints
resource = model3.addConstrs((x.sum(r,'*') <= 1 for r in R), 'resource')

$$
\sum_{j \: \in \: J} x_{r,j} \leq 1 \; \; \; \forall \; r \in R
$$

### Setting the objective function

The objective function is to maximize the total matching score of the assignments.

$$
\max \; (53x_{1,1} + 80x_{2,1} + 53x_{3,1})
 + (27x_{1,2} + 47x_{2,2} + 73x_{3,2})
 + (13x_{1,3} + 67x_{2,3} + 47x_{3,3})
$$

The “setObjective()” method defines the objective function of the model object “model”. 

In [ ]:
# The objective is to maximize total matching score of the assignments
model3.setObjective(x.prod(ms), GRB.MAXIMIZE)

Notice that 
$$
(53x_{1,1} + 80x_{2,1} + 53x_{3,1}) = \sum_{r \; \in \; R} ms_{r,1}x_{r,1} \\
(27x_{1,2} + 47x_{2,2} + 73x_{3,2}) = \sum_{r \; \in \; R} ms_{r,2}x_{r,2} \\
(13x_{1,3} + 67x_{2,3} + 47x_{3,3})  = \sum_{r \; \in \; R} ms_{r,3}x_{r,3}
$$

Hence, the objective function can be expressed as follows

$$
\max \; \sum_{j \; \in \; J} \sum_{r \; \in \; R} ms_{r,j}x_{r,j}
$$

### Save model (.lp or .mps)

In [ ]:
# save model for inspection
model3.write('model3.lp')

### Optimizing the model

In [ ]:
# run optimization engine
model3.optimize()

### Reporting the results

In [ ]:
def print_solution(model):
    for var in model3.getVars():
        if abs(var.x) > 1e-6:
            print("{0}: {1}".format(var.varName, var.x))

    print('Total matching score: {0}'.format(model3.objVal))
    print('Runtime: {0}'.format(model3.Runtime))
    return None



In [ ]:
# display optimal values of decision variables
print_solution(model3)   

## Refererences:

https://pypi.org/project/gurobipy/

https://support.gurobi.com/hc/en-us/categories/360000840931-Getting-Started-with-Gurobi

https://support.gurobi.com/hc/en-us/articles/360044290292-How-do-I-install-Gurobi-for-Python-

https://support.gurobi.com/hc/en-us/articles/17278438215313-Tutorial-Getting-Started-with-the-Gurobi-Python-API
